# Machine learning homework 6

## 0 Preparation

### 0.1 Import required librarys

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import display, Image

### 0.2 Read data from files

In [ ]:
circle = np.loadtxt('data/circle.txt', delimiter=',')
moon = np.loadtxt('data/moon.txt', delimiter=',')

### 0.3 Take a look at the data

In [ ]:
def scatter_animation(x, y, color_steps, save):
    def update_scatter(frame, scat, color_steps):
        scat.set_array(color_steps[:, frame])
        return scat,
    fig, ax = plt.subplots()
    # Must initialize with that number of cluster!!
    scat = ax.scatter(x, y, c=color_steps[:, -1])
    animation = ani.FuncAnimation(fig, update_scatter, frames=color_steps.shape[1],
                                  blit=True, interval=300, repeat_delay=500, fargs=(scat, color_steps))
    plt.close()
    animation.save(save, writer='imagemagick')
    display(Image(filename=save))

In [ ]:
def plot_graph(x1, x2, y1=None, y2=None, title='', animated=False):
    """Plot circle and moon, with or without label, animate or not."""
    y1_final = y1[:, -1] if animated else y1
    y2_final = y2[:, -1] if animated else y2
    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
    fig.suptitle(title)
    ax[0].set_title('circle')
    ax[0].scatter(x1[:, 0], x1[:, 1], c=y1_final)
    ax[1].set_title('moon')
    ax[1].scatter(x2[:, 0], x2[:, 1], c=y2_final)
    plt.show()
    if animated:
        scatter_animation(x1[:, 0], x1[:, 1], y1, f'{title}_circle.gif')
        scatter_animation(x2[:, 0], x2[:, 1], y2, f'{title}_moon.gif')
plot_graph(circle, moon)

## 1 Make animation to show clustering procedure

### 1.1 k-means

#### Define initialization method for k-means
This method randomly assign each sample to one cluster

In [ ]:
def random_assign(x, n_class):
    """Initialization method which random assign class."""
    y = np.random.randint(0, n_class, x.shape[0])
    return y

#### Define E step of k-means

In [ ]:
def assign_cluster(x, means):
    """Assign x to different cluster according to closest mean."""
    # Add a new axis to x
    x_new = x[:, None, :]
    # Compute euclidean distance for each pair of x and mean point
    distance = np.sqrt(np.sum((x_new - means)**2, axis=2))
    # Assign cluster to a point with smallest distance from means
    y = np.argmin(distance, axis=1)
    return y

#### Define M step of k-means

In [ ]:
def compute_means(x, y, k):
    """Compute the mean value for each cluster of x."""
    means = np.array([x[y == c].mean(axis=0) for c in range(k)])
    return means

#### Combine E and M step to perform k-means

In [ ]:
def kmeans(x, k=2, init_method=random_assign, return_step=False):
    """Perform k-means clustering on x."""
    # Randomly assign cluster to each x as initialization
    y = init_method(x, k)
    steps = y[:, None].copy()
    means = compute_means(x, y, k)
    prev_means = means
    # Repeatly assign cluster and compute new means until converge
    while True:
        y = assign_cluster(x, means)
        means = compute_means(x, y, k)
        steps = np.hstack((steps, y[:, None]))
        # Stop iteration if means didn't move
        if np.allclose(means, prev_means):
            break
        prev_means = means
    return y if not return_step else steps

#### Perform k-means on circle and moon

In [ ]:
y_circle = kmeans(circle, return_step=True)
y_moon = kmeans(moon, return_step=True)
plot_graph(circle, moon, y_circle, y_moon, 'k-means', animated=True)

### 1.2 Kernel k-means

#### Define kernel function
Here we use RBF kernel

In [ ]:
def rbf_kernel(x1, x2, gamma = 5):
    """Custom kernel function which accept two list of samples and return a list of result."""
    euclidean = np.sqrt(np.sum((x1 - x2) ** 2, axis=1))
    rbf = np.exp(-gamma * euclidean ** 2)
    return rbf

#### Define function for precomputing data using kernel

In [ ]:
def precomputed(x1, x2, kernel_func):
    """Precomputed x1 with x2 using kernel function, return array of shape (n_x1, n_x2)."""
    result = np.zeros((x1.shape[0], x2.shape[0]))
    for j in range(result.shape[1]):
        x2_j = x2[j][None, :]
        result[:, j] = kernel_func(x1, x2_j)
    return result

### Define main function for kernel k-means

In [ ]:
def kernel_kmeans(x, k=2, init_method=random_assign, return_step=False):
    """Perform kernel k-means clustering on x."""
    # Compute gram matrix
    gram_mat = precomputed(x, x, rbf_kernel)
    # Construct array of size (n_sample, k) to store distance
    distance = np.zeros((x.shape[0], k))
    # Randomly assign cluster to each x as initialization
    y = init_method(x, k)
    steps = y[:, None].copy()
    prev_y = y
    while True:
        # Compute all sample to each center's distance
        for cluster in range(k):
            mask = (y == cluster)
            first = gram_mat.diagonal()
            second = -2 * (mask * gram_mat).sum(axis=1) / mask.sum()
            third = (mask * (mask * gram_mat).sum(axis=1)).sum() / (mask.sum() ** 2)
            distance[:, cluster] = first + second + third
        # Pick the cluster with smallest distance
        y = np.argmin(distance, axis=1)
        steps = np.hstack((steps, y[:, None]))
        # Stop if y didn't change anymore
        if (prev_y == y).all():
            break
        prev_y = y
    return y if not return_step else steps

#### Perform kernel k-means on circle and moon

In [ ]:
y_circle = kernel_kmeans(circle, return_step=True)
y_moon = kernel_kmeans(moon, return_step=True)
plot_graph(circle, moon, y_circle, y_moon, 'kernel k-means', animated=True)

### 1.3 Spectral clustering
Reference:
- https://towardsdatascience.com/spectral-clustering-82d3cff3d3b7

#### Define main function for spectral clustering

In [ ]:
def spectral_clustering(x, k=2, gamma=50, init_method=random_assign, return_step=False):
    distance_mat = np.sqrt(np.sum((x[:, None, :] - x) ** 2, axis=2))
    adjacency_mat = np.exp(-gamma * (distance_mat ** 2))
    degree_mat = np.diag(adjacency_mat.sum(axis=1))
    graph_laplacian = degree_mat - adjacency_mat
    eigen_val, eigen_vec = np.linalg.eig(graph_laplacian)
    eigen_vec = eigen_vec[:,np.argsort(eigen_val)]
    eigen_val = eigen_val[np.argsort(eigen_val)]
    y = kmeans(eigen_vec[:, 0:k], k=k, init_method=init_method, return_step=return_step)
    return y

#### Perform spectral clustering on circle and moon

In [ ]:
y_circle = spectral_clustering(circle, return_step=True)
y_moon = spectral_clustering(moon, return_step=True)
plot_graph(circle, moon, y_circle, y_moon, 'spectral clustering', animated=True)

### 1.4 DBSCAN
Reference:
- https://en.wikipedia.org/wiki/DBSCAN

#### Define main function for DBSCAN

In [ ]:
def dbscan(x, epsilon=0.15, min_points=3, return_step=False, step_resolution=50):
    """Perform DBSCAN clustering on x."""
    def find_neighbors(distance, radius=epsilon):
        """Return index of distance which is smaller than radius."""
        return np.argwhere(distance < radius).flatten()
    cluster = 0
    # Initialize label of each data point(0:unlabelled, -1:noise, >0:cluster)
    y = np.zeros(x.shape[0], dtype=int)
    steps = y[:, None].copy()
    # Compute distance matrix using broadcasting in numpy
    distance_mat = np.sqrt(np.sum((x[:, None, :] - x) ** 2, axis=2))
    # Iterate each data point
    for i in range(x.shape[0]):
        # Skip labelled data point
        if y[i] != 0:
            continue
        # Find index of neighbor which has distance smaller than epsilon from current point
        neighbors_idx = find_neighbors(distance_mat[i])
        # Mark as noise due to not enough neighbors
        if neighbors_idx.shape[0] < min_points:
            y[i] = -1
            continue
        # Assign new cluster
        cluster += 1
        y[i] = cluster
        # Visit all neighbors and corresponding neighbors
        neighbors_idx = neighbors_idx[neighbors_idx != i]
        while neighbors_idx.shape[0] > 0:
            idx = neighbors_idx[0]
            # Process unlabelled or noise neighbor
            if y[idx] <= 0:
                # Find unlabelled neighbor's neighbors
                if y[idx] == 0:
                    new_neighbors_idx = find_neighbors(distance_mat[idx])
                    if new_neighbors_idx.shape[0] >= min_points:
                        neighbors_idx = np.append(neighbors_idx, new_neighbors_idx)
                # Assign current cluster
                y[idx] = cluster
                steps = np.hstack((steps, y[:, None]))
            neighbors_idx = neighbors_idx[1:]
    # Map label back to 0 start
    steps[steps == steps.max()] = 0
    return y - 1 if not return_step else steps[:, ::step_resolution]

#### Perform DBSCAN on circle and moon

In [ ]:
y_circle = dbscan(circle)
y_moon = dbscan(moon)

In [ ]:
y_circle = dbscan(circle, return_step=True)
y_moon = dbscan(moon, return_step=True)
plot_graph(circle, moon, y_circle, y_moon, 'DBSCAN', animated=True)

## 2 Different k

### 2.1 k-means

In [ ]:
k_to_try = [2, 3, 4]
for k in k_to_try:
    y_circle = kmeans(circle, k=k)
    y_moon = kmeans(moon, k=k)
    plot_graph(circle, moon, y_circle, y_moon, f'k-means, k={k}')

### 2.2 Kernel k-means

In [ ]:
for k in k_to_try:
    y_circle = kernel_kmeans(circle, k=k)
    y_moon = kernel_kmeans(moon, k=k)
    plot_graph(circle, moon, y_circle, y_moon, f'kernel k-means, k={k}')

### 2.3 Spectral clustering

In [ ]:
for k in k_to_try:
    y_circle = spectral_clustering(circle, k=k)
    y_moon = spectral_clustering(moon, k=k)
    plot_graph(circle, moon, y_circle, y_moon, f'spectral clustering, k={k}')

### 2.4 DBSCAN
DBSCAN don't need the parameter `k`, it will automatically find appropriate number of groups to cluter. However, it has parameter $\epsilon$ and `min_pts` which need to be set carefully to obtain the best result.

## 3 Different initialization method
Here I use k-means++ as alternate initialization method

### Define the k-means++ initialization method

In [ ]:
def kmpp_assign(x, n_class):
    """Initialization method which use k-means++ for assigning class."""
    # Find n means from x
    means = None
    for c in range(n_class):
        if means is None:
            mean = x[np.random.choice(x.shape[0])]
            means = mean[None, :]
            continue
        distance_to_means = np.sqrt(np.sum(((x[:, None, :] - means) ** 2), axis=2))
        closest_distance_to_means = distance_to_means.min(axis=1)
        probability = closest_distance_to_means / closest_distance_to_means.sum()
        mean = x[np.random.choice(x.shape[0], p=probability)]
        means = np.vstack((means, mean))
    y = assign_cluster(x, means)
    return y

In [ ]:
y_circle = kmeans(circle, init_method=kmpp_assign)
y_moon = kmeans(moon, init_method=kmpp_assign)
plot_graph(circle, moon, y_circle, y_moon, 'k-means')

In [ ]:
y_circle = kernel_kmeans(circle, init_method=kmpp_assign)
y_moon = kernel_kmeans(moon, init_method=kmpp_assign)
plot_graph(circle, moon, y_circle, y_moon, 'kernel k-means')

In [ ]:
y_circle = spectral_clustering(circle, init_method=kmpp_assign)
y_moon = spectral_clustering(moon, init_method=kmpp_assign)
plot_graph(circle, moon, y_circle, y_moon, 'spectral clustering')

## 4 Eigen space visualization in spectral clustering